In [2]:
import os
import torch
import torchvision.transforms as transforms
import albumentations as A
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from LookGenerator.datasets.segmentation_generator_dataset import SegGenDataset
from LookGenerator.networks.encoder_decoder import EncoderDecoder
from LookGenerator.networks.losses import FocalLoss
from LookGenerator.networks.trainer import Trainer
from LookGenerator.networks.utils import load_model
from LookGenerator.networks_training.utils import check_path_and_creat
from torchvision.transforms import ToTensor, ToPILImage

In [ ]:
toten = ToTensor()
topil = ToPILImage()

In [3]:
augment = A.Resize(256,192)
batch_size = 1

In [4]:
train_dataset = SegGenDataset(image_dir=r"C:\Users\Даша\кто\мусор\zalando-hd-resized\train",
                        augment=augment)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

In [6]:
for input_, target in train_dataloader:
    input_ = input_.squeeze(0)
    pic = transforms.ToPILImage()(input_[:3])
    pic.show()
    for i in range(3,11):
        pic = transforms.ToPILImage()(input_[i])
        pic.show()
    break

In [ ]:
val_dataset = SegGenDataset(image_dir=r"",
                        augment=augment)
val_dataloader = DataLoader(train_dataset, batch_size=batch_size)

In [ ]:
model = EncoderDecoder(in_channels=16,
                       out_channels=3)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
criterion = FocalLoss()
print(device)

In [ ]:
save_directory = r'C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\segmentationMulty\weights\testMulty_out_12_6features_noGauss'
check_path_and_creat(save_directory)

In [ ]:
trainer = Trainer(
    model_=model, optimizer=optimizer, criterion=criterion,
    device=device, save_directory=save_directory, save_step=1, verbose=True
)

In [ ]:
epoch_num=80

In [ ]:
trainer.train(train_dataloader=train_dataloader, val_dataloader=val_dataloader, epoch_num=epoch_num)

In [ ]:
trainer.draw_history_plots()

In [ ]:
trainer.save_history_plots(save_directory)

わたしの最後はあなたがいい

In [ ]:
model = load_model(model, r'C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\segmentationMulty\weights\testMulty_out_12_6features_noGauss\epoch_39.pt')

In [ ]:
test_dataset = SegGenDataset(image_dir=r"",
                        augment=augment)
test_dataloader = DataLoader(test_dataset, batch_size=1)

In [ ]:
for input_, target in test_dataloader:
    result = model(input_)

    result = result.squueze(0)

    fig, axes = plt.subplots(2, 3)

    for i in range(3):
        axes[0,i].imshow(np.array(topil(result[i])), cmap = 'gray')
        axes[1,i]. imshow(np.array(topil(target[i])), cmap = 'gray')
    break